# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [14]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import gmaps.datasets
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [4]:
cities_df = pd.read_csv("../Output/weather_data.csv")
cities_df.head()


,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,Karratha,-20.74,116.85,301.02,18,0,8.91,AU,1598060623
1,1,Punta Arenas,-53.15,-70.92,278.71,80,100,10.30,CL,1598060467
2,2,Marsabit,2.33,37.99,288.14,97,100,9.37,KE,1598060625
3,3,Ushuaia,-54.80,-68.30,278.15,80,75,2.10,AR,1598060622
4,4,Cape Town,-33.93,18.42,280.37,93,0,2.10,ZA,1598060060


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [23]:
# Configure gmaps
gmaps.configure(api_key=g_key)

# Define humidity
locations = cities_df[["Lat", "Lng"]]
humidity = cities_df["Humidity"].astype(float)

In [24]:
# Create heatmap of humidity
fig = gmaps.figure(map_type='HYBRID')
heat_layer = gmaps.heatmap_layer(locations, 
                                 weights = humidity,
                                 dissipating = False, 
                                 max_intensity = 100,
                                 point_radius = 3)
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [29]:
narrowed_df = cities_df.loc[(cities_df["Wind Speed"] <= 10) & (cities_df["Cloudiness"] == 0) & \
                            (cities_df["Max Temp"] >= 285) & (cities_df["Max Temp"] <= 300)].dropna()

narrowed_df

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
14,14,Mayya,61.73,130.28,289.15,41,0,2.00,RU,1598060631
23,23,Saint-Pierre,-21.34,55.48,290.15,59,0,4.10,RE,1598060637
27,27,Ząbkowice Śląskie,50.59,16.81,294.82,87,0,3.15,PL,1598060639
28,28,Lagoa,39.05,-27.98,293.84,65,0,1.63,PT,1598060443
65,65,Legnago,45.18,11.31,297.59,86,0,2.24,IT,1598060665
86,86,Watrous,51.67,-105.47,293.62,66,0,2.74,CA,1598060678
103,103,Constitución,-35.33,-72.42,285.14,79,0,1.58,CL,1598060689
111,111,Barcelos,41.54,-8.62,290.93,93,0,1.50,PT,1598060512
128,128,Borogontsy,62.67,131.17,289.37,37,0,1.40,RU,1598060703
130,130,Gurlan,41.84,60.39,291.15,52,0,2.10,UZ,1598060704


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [30]:
# Store into hotel_df
hotel_df = narrowed_df.loc[:,["City","Country", "Lat", "Lng"]]

# Add new column
hotel_df["Hotel Name"] = ""

# Display result
hotel_df

,City,Country,Lat,Lng,Hotel Name
14,Mayya,RU,61.73,130.28,
23,Saint-Pierre,RE,-21.34,55.48,
27,Ząbkowice Śląskie,PL,50.59,16.81,
28,Lagoa,PT,39.05,-27.98,
65,Legnago,IT,45.18,11.31,
86,Watrous,CA,51.67,-105.47,
103,Constitución,CL,-35.33,-72.42,
111,Barcelos,PT,41.54,-8.62,
128,Borogontsy,RU,62.67,131.17,
130,Gurlan,UZ,41.84,60.39,


In [40]:
# Retrieve URL and define parameters
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

params = {"type" : "hotel",
          "keyword" : "hotel",
          "radius" : 5000,
          "key" : g_key}

In [41]:
hotel_list = []

# Start loop

for index, row in hotel_df.iterrows():
    lat = row["Lat"]
    lng = row["Lng"]
    city_name = row["City"]
    
    params["location"] = f"{lat},{lng}"

    # Make API call
    response = requests.get(base_url, params=params).json()
    
    # Obtin results
    results = response['results']
    
    # Append names to hotel_df
    try:
        hotel_df.loc[index, "Hotel Name"] = results[0]['name']

    # if there is no hotel available, show missing field
    except (KeyError, IndexError):
        print("No hotels found.. skipping")
        
    print("------------")

# Print end of search once searching is completed
print("Search completed")

------------
------------
------------
------------
------------
------------
------------
------------
------------
No hotels found.. skipping
------------
------------
------------
------------
No hotels found.. skipping
------------
------------
------------
------------
No hotels found.. skipping
------------
------------
------------
------------
------------
------------
------------
------------
------------
No hotels found.. skipping
------------
------------
------------
------------
No hotels found.. skipping
------------
------------
------------
------------
------------
------------
------------
------------
------------
------------
------------
------------
------------
------------
------------
------------
------------
No hotels found.. skipping
------------
------------
------------
------------
------------
------------
No hotels found.. skipping
------------
------------
------------
------------
------------
------------
Search completed


In [43]:
hotel_df.head()

,City,Country,Lat,Lng,Hotel Name
14,Mayya,RU,61.73,130.28,Sberbank Rossii
23,Saint-Pierre,RE,-21.34,55.48,LODGE PALMAE
27,Ząbkowice Śląskie,PL,50.59,16.81,HOTEL - RESTAURACJA DOLNOŚLĄSKA
28,Lagoa,PT,39.05,-27.98,Graciosa Resort - Biosphere Island Hotel
65,Legnago,IT,45.18,11.31,Villa Brama


In [44]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [45]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations)

# Display figure
fig.add_layer(markers)
fig

Figure(layout=FigureLayout(height='420px'))